In [ ]:
"""
Generic Rate Limiter 
"""
import time 
from collections import defaultdict, deque 
class RateLimiter:
    def __init__(self, max_requests: int, window_seconds: int) -> None:
        self.max_requests = max_requests
        self.window = window_seconds
        self.requests = defaultdict(deque)
    
    def allow_request(self, user_id: str) -> bool:
        now = time.time()
        q = self.requests[user_id]

        while q and now-q[0]>=self.window:
            q.popleft()
        if len(q)<self.max_requests:
            q.append(now)
            return True 
        return False 

limiter = RateLimiter(max_requests=3, window_seconds=10)

assert limiter.allow_request("alice") == True   # 1st
assert limiter.allow_request("alice") == True   # 2nd
assert limiter.allow_request("alice") == True   # 3rd
assert limiter.allow_request("alice") == False  # 4th - denied
assert limiter.allow_request("bob") == True     # bob's 1st
                    

In [ ]:
"""
Token Bucket Rate Limiter 
"""
import datetime 
from collections import deque 
class TokenBucketRateLimiter:
    def __init__(self, capacity: int, refill_rate: float) -> None:
        self.capacity = capacity
        self.refill_rate = refill_rate
        self.requests = deque([])
        self.tokens = self.capacity
        
    
    def allow_request(self, user_id: str) -> bool:
        now = datetime.timedelta()
        if self.requests:
            period = (now - self.requests[-1][1]).total_seconds()
            token_add = period*self.refill_rate
            if token_add+self.tokens > self.capacity:
                self.tokens = self.capacity
            else:
                self.tokens+=token_add
        if self.tokens>=1:
            self.requests.append((user_id, datetime.timedelta()))
            self.tokens-=1
        else:
            print("DENIED ( no tokens )")





In [1]:
"""
Test Token Bucket Algorithm Unit Case
"""
import time
class TokenBucketRateLimiter:
    def __init__(self, capacity: int, refill_rate: float) -> None:
        # per-user buckets
        self.capacity = capacity
        self.refill_rate = refill_rate  # tokens per second
        self.users = {}  # user_id -> {tokens, last_refill_time}

    def allow_request(self, user_id: str) -> bool:
        now = time.time()

        if user_id not in self.users:
            self.users[user_id] = {
                "tokens": self.capacity,
                "last_refill": now
            }

        bucket = self.users[user_id]

        # Refill tokens based on elapsed time
        elapsed = now - bucket["last_refill"]
        refill_amount = elapsed * self.refill_rate

        bucket["tokens"] = min(self.capacity, bucket["tokens"] + refill_amount)
        bucket["last_refill"] = now

        # Accept or reject request
        if bucket["tokens"] >= 1:
            bucket["tokens"] -= 1
            return True
        else:
            return False


# ------------------------------
# Test Script (Visual Example)
# ------------------------------
def print_state(limiter, user):
    """Helper: print remaining tokens."""
    tokens = limiter.users[user]["tokens"]
    print(f"[{user}] tokens left: {tokens:.2f}")

def run_test():
    print("---- TOKEN BUCKET RATE LIMITER TEST ----")
    limiter = TokenBucketRateLimiter(capacity=5, refill_rate=2)

    user = "alice"

    # Use all 5 tokens
    print("\nUsing all tokens:")
    for i in range(1, 7):
        allowed = limiter.allow_request(user)
        print(f"Request {i}: {'ALLOWED' if allowed else 'DENIED'}")
        print_state(limiter, user)
        time.sleep(0.1)  # small delay just for display consistency

    # Wait 1 second → 2 tokens refill
    print("\nWaiting 1 second for refill...")
    time.sleep(1)

    print("Making 2 more requests:")
    allowed = limiter.allow_request(user)
    print("Request after 1s #1:", "ALLOWED" if allowed else "DENIED")
    print_state(limiter, user)

    allowed = limiter.allow_request(user)
    print("Request after 1s #2:", "ALLOWED" if allowed else "DENIED")
    print_state(limiter, user)

    # Final assert tests (strict correctness)
    print("\nRunning assert checks...")
    limiter2 = TokenBucketRateLimiter(capacity=5, refill_rate=2)

    assert limiter2.allow_request("alice") == True
    assert limiter2.allow_request("alice") == True
    assert limiter2.allow_request("alice") == True
    assert limiter2.allow_request("alice") == True
    assert limiter2.allow_request("alice") == True
    assert limiter2.allow_request("alice") == False

    print("All asserts passed.")

    print("\n--- TEST COMPLETE ---")


if __name__ == "__main__":
    run_test()


---- TOKEN BUCKET RATE LIMITER TEST ----

Using all tokens:
Request 1: ALLOWED
[alice] tokens left: 4.00
Request 2: ALLOWED
[alice] tokens left: 3.21
Request 3: ALLOWED
[alice] tokens left: 2.42
Request 4: ALLOWED
[alice] tokens left: 1.62
Request 5: ALLOWED
[alice] tokens left: 0.83
Request 6: ALLOWED
[alice] tokens left: 0.04

Waiting 1 second for refill...
Making 2 more requests:
Request after 1s #1: ALLOWED
[alice] tokens left: 1.25
Request after 1s #2: ALLOWED
[alice] tokens left: 0.25

Running assert checks...
All asserts passed.

--- TEST COMPLETE ---
